In [2]:
# importing relevant libraries
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
import time
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: use options instead of chrome_options


In [15]:
def request_page(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    parsed_list_of_ads = soup.find('section', attrs={'class': 'a-search-container main-cols-container'})
    total_ad_number = get_total_number_ads(parsed_list_of_ads)
    number_of_pages = int(total_ad_number/20) + 1

    print("The total number of apartments found by search is " + str(total_ad_number))
    print("The total number of pages found by search is " + str(number_of_pages))
    return number_of_pages

# The function below gets the total number of ads
def get_total_number_ads(parsed_list):
    text = parsed_list.find(class_="a-search-subtitle search-results-nb").get_text()
    res = [int(i) for i in text.split() if i.isdigit()] 
    total_number = res[0]*1000+res[1]
    return total_number

In [11]:
# The function tokenize is needed to get the list of words from web scraping 

def tokenize(content):
    splitter = [" ", "\n", ",", ")", "\"", "–", "-", "/", "\xa0"]
    result_list = []
    current_word = ""
    for x in content:
        if x in splitter:
            if current_word != "":
                result_list.append(current_word)
            current_word = ""
        else:
            current_word += x
            
    return result_list

In [13]:
# getting the whole text from the specific page
def get_soup_spec_page(url, page_number):
    URL = url+'?das[checked]=1&das[who]=1&page=' + str(page_number)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    full_list = soup.find('section', attrs={'class': 'a-list a-search-list a-list-with-favs'})
    return full_list

In [14]:
def get_df_list(full_list):
    df_apartments = pd.DataFrame(columns=['number_of_rooms', 'area', 'storey_number', 
                                          'total_storey_number', 'price', 'district', 
                                          'complex_name', 'year', 'condition', 'phone_nums'])
    adv_elems = full_list.find_all(True, {'class':['a-card__descr']})
    for elem in adv_elems:
        data_id = elem.find('a', class_='a-card__title').get_text()
        number_of_bedrooms_with_area_draft = elem.find('a', class_='a-card__title').get_text()
        number_of_bedrooms_with_area = tokenize(number_of_bedrooms_with_area_draft)
        number_of_rooms = int(number_of_bedrooms_with_area[0])
        area = float(number_of_bedrooms_with_area[3])
        ahref = elem.find('a', class_='a-card__title')['href']
        driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

        url = "https://krisha.kz"+ahref
        driver.get(url)
      
        button = driver.find_element_by_class_name("show-phones")
        button.click()
        time.sleep(0.5)
        tels = driver.find_element_by_class_name("offer__contacts-phones")
        phone_nums = str(tels.text)
      
        if len(number_of_bedrooms_with_area) > 5:
            storey_number = number_of_bedrooms_with_area[5]
            if len(number_of_bedrooms_with_area) < 7:
                total_storey_number = np.nan
            else:
                total_storey_number = number_of_bedrooms_with_area[6]
        else:
            storey_number = np.nan
            total_storey_number = np.nan
        
        
        price_draft = tokenize(elem.find('div', class_='a-card__price').get_text())
        
        if price_draft[0].isnumeric() and price_draft[1].isnumeric() and price_draft[2].isnumeric():
            price = int(price_draft[0]+price_draft[1]+price_draft[2])
        elif price_draft[1].isnumeric() and price_draft[2].isnumeric() and price_draft[3].isnumeric():
            price = int(price_draft[1]+price_draft[2]+price_draft[3])
        else:
            price = np.nan
            
        district_draft = elem.find('div', class_='a-card__subtitle').get_text()
        district_draft = tokenize(district_draft)
        district = np.nan
        if len(district_draft)>1 and district_draft[1] == 'р' and district_draft[2] == 'н':
            district = district_draft[0]
        complex_name_draft = elem.find('div', class_ = 'a-card__text-preview').get_text()
        complex_name_draft = tokenize(complex_name_draft)
        year = np.nan
        for i in range(11):
            if len(complex_name_draft[i]) == 4 and complex_name_draft[i].isdigit():
                year = int(complex_name_draft[i])
                break
        apartment_type_list = ['кирпичный', 'монолитный', 'панельный']
        complex_name = np.nan
        if complex_name_draft[0] == 'жил.':
            complex_name = complex_name_draft[2]
            if complex_name_draft[3].isnumeric() == False and complex_name_draft[3] not in apartment_type_list:
                complex_name += ' '
                complex_name += complex_name_draft[3]
        condition = np.nan
        if 'состояние:' in complex_name_draft:
            index = complex_name_draft.index('состояние:')
            condition = complex_name_draft[index+1]
        df_apartments.loc[len(df_apartments.index)] = [number_of_rooms, area, storey_number, total_storey_number, price, district, complex_name, year, condition, phone_nums]
    return df_apartments

In [4]:
urls = ['https://krisha.kz/prodazha/', 'https://krisha.kz/arenda/']

In [5]:
submenu = ['kvartiry/', 'doma/', 'dachi/', 'uchastkov/', 'ofisa/', 'pomeshhenija/', 'zdanija/', 'magazina/', 'prombazy/', 'prochej-nedvizhimosti/']

In [19]:
for i in range(len(urls)):
  for j in range(len(submenu)):
    number_of_pages = request_page(urls[i]+submenu[j])
    df = pd.DataFrame(columns=['number_of_rooms', 'area', 'storey_number', 'total_storey_number', 'price', 'district', 'complex_name', 'year', 'condition', 'phone_nums'])
    for k in range(number_of_pages):
        df = df.append(get_df_list(get_soup_spec_page(urls[i]+submenu[j], k+1)), ignore_index = True)

df


The total number of apartments found by search is 156934
The total number of pages found by search is 7847


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]
/usr/local/li

   number_of_rooms    area  ...   condition                        phone_nums
0                3   60.00  ...     среднее                   +7 775 517 7257
1                3   90.60  ...         NaN                   +7 778 021 1807
2                1   44.60  ...         NaN                   +7 778 021 1867
3                2   44.00  ...     хорошее  +7 701 934 3848\n+7 707 297 6141
4                2   53.50  ...     хорошее                   +7 777 176 3333
5                3  117.20  ...         NaN                   +7 701 544 8144
6                2   98.00  ...  евроремонт                   +7 708 351 2404
7                3   58.00  ...  евроремонт                   +7 701 925 4446
8                2   59.40  ...  евроремонт                   +7 747 603 8939
9                3   65.00  ...     хорошее                   +7 777 196 8013
10               4  238.00  ...         NaN                   +7 705 900 0832
11               2   54.00  ...     среднее                   +7

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


KeyboardInterrupt: ignored